## 1. Import Libs
- Unpaired Dataset과 Cycle Consistency를 사용
- Domain Transfer를 목적으로 간단한 Network를 사용함
- CycleGAN에 비해 높은 해상도 보단 형태 변화에 집중

In [1]:
import argparse
import os
import numpy as np
import math
import itertools
import sys
import datetime
import time
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision.transforms as transforms
from torchvision.utils import save_image

from models import *
from datasets import *

CUDA = torch.cuda.is_available()

## 2. Utils

In [2]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if(classname.find('Conv') != -1):
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)
            
#save results for every few epochs
def sample_images(epoch_num, test_data):
    imgs = next(iter(test_data))
    real_A = imgs['A'].cuda()
    fake_B = G_AB(real_A)
    real_B = imgs['B'].cuda()
    fake_A = G_BA(real_B)
    fin = torch.cat((real_A.data, fake_B.data, real_B.data, fake_A.data), 0)
    save_image(fin, 'results/%s.png' % (epoch_num), nrow=8, normalize=True)

## 3. Load Dataset
### 2.1 Preprocessing

In [3]:
data_dir = 'data/edges2shoes'

transform = [
    transforms.Resize((64, 64), Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, .5), (.5, .5, .5))
]

### 2.2 Data Load

In [4]:
traindata = ImageDataset("data/edges2shoes", transforms_=transform, mode='train')
trainLoader = DataLoader(traindata, batch_size=16, shuffle=False, num_workers=4)

testdata = ImageDataset("data/edges2shoes", transforms_=transform, mode='val')
testLoader = DataLoader(testdata, batch_size=4, shuffle=True, num_workers=4)

test_real_train_data = traindata.__getitem__(11)
print(test_real_train_data)
print(len(test_real_train_data))

{'A': tensor([[[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
         [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
         [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
         ...,
         [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
         [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
         [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000]],

        [[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
         [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
         [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
         ...,
         [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
         [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
         [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000]],

        [[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
         [ 1.0000,  1.0000,  1.0000,  .

## 4. Build Models & Optimizers & Loss Function
### 4.1 Models

In [5]:
G_AB = GeneratorUNet()
G_BA = GeneratorUNet()
D_A = Discriminator()
D_B = Discriminator()

print(G_AB)
print(G_BA)
print(D_A)
print(D_B)

GeneratorUNet(
  (down1): UNetDown(
    (model): Sequential(
      (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.2)
    )
  )
  (down2): UNetDown(
    (model): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
  )
  (down3): UNetDown(
    (model): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
      (3): Dropout(p=0.5)
    )
  )
  (down4): UNetDown(
    (model): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): LeakyReL

### 4.2 Optimizers

In [6]:
optimizer_G = torch.optim.Adam(itertools.chain(G_AB.parameters(), G_BA.parameters()), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=0.0002, betas=(0.5, 0.999))

### 4.3 Loss Functions

In [7]:
#loss functions
adversarial_loss = nn.MSELoss() # To check how well our generators are doing against our discriminators and vice-versa
cycle_loss = nn.L1Loss() # Reconstruction loss
pixelwise_loss = nn.L1Loss() # To compare fake images with real images

if(CUDA):
    G_AB = G_AB.cuda()
    G_BA = G_BA.cuda()
    D_A = D_A.cuda()
    D_B = D_B.cuda()
    adversarial_loss.cuda()
    cycle_loss.cuda()
    pixelwise_loss.cuda()
    
G_AB.apply(weights_init_normal)
G_BA.apply(weights_init_normal)
D_A.apply(weights_init_normal)
D_B.apply(weights_init_normal)

avg_losses_G = []
avg_losses_D_A = []
avg_losses_D_B = []

## 5. Training

In [8]:
for epoch in range(1, 200 + 1):
    for i, batch in enumerate(trainLoader, 1):
        real_A = batch['A'].cuda()
        real_B = batch['B'].cuda()
        
        valid = torch.ones(real_A.size(0), 1, requires_grad=False).cuda()
        fake = torch.zeros(real_B.size(0), 1, requires_grad=False).cuda()
        
        #### train Generators
        optimizer_G.zero_grad()
        
        # GAN loss
        fake_B = G_AB(real_A)
        loss_GAN_AB = adversarial_loss(D_B(fake_B), valid)
        fake_A = G_BA(real_B)
        loss_GAN_BA = adversarial_loss(D_A(fake_A), valid)
        
        loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2
        
        # pixelwise translation loss
        loss_pixelwise = (pixelwise_loss(fake_B, real_B) + pixelwise_loss(fake_A, real_A)) / 2
        
        # cycle loss
        loss_cycle_A = cycle_loss(G_BA(fake_B), real_A)
        loss_cycle_B = cycle_loss(G_AB(fake_A), real_B)
        loss_cycle = (loss_cycle_A + loss_cycle_B) / 2
        
        # total loss
        loss_G = loss_GAN + loss_pixelwise + loss_cycle
        loss_G.backward()
        optimizer_G.step()
        
        #### Train Discriminators
        optimizer_D_A.zero_grad()
        
        loss_real = adversarial_loss(D_A(real_A), valid)
        loss_fake = adversarial_loss(D_A(fake_A.detach()), fake)
        
        loss_D_A = (loss_real + loss_fake) / 2
        loss_D_A.backward()
        optimizer_D_A.step()
        
        optimizer_D_B.zero_grad()
        
        loss_real = adversarial_loss(D_B(real_B), valid)
        loss_fake = adversarial_loss(D_B(fake_B.detach()), fake)
        
        loss_D_B = (loss_real + loss_fake) / 2
        loss_D_B.backward()
        optimizer_D_B.step()
        
        if i%100 == 0:
            print('Epoch [%d/%d], Step [%d/%d], loss_D_A: %.4f, loss_D_B: %.4f, loss_G: %.4f'
                  % (epoch, 200, i, len(trainLoader), loss_D_A.data[0], loss_D_B.data[0], loss_G.data[0]))
        
    sample_images(epoch, testLoader)
    
    if epoch % 50 == 0:
       # Save model checkpoints
        torch.save(G_AB.state_dict(), 'save_model/G_AB.pkl')
        torch.save(G_BA.state_dict(), 'save_model/G_BA.pkl')
        torch.save(D_A.state_dict(), 'save_model/D_A.pkl')
        torch.save(D_B.state_dict(), 'save_model/D_B.pkl')

/home/ncp/anaconda3/envs/pytorch0_4/lib/python3.6/site-packages/ipykernel_launcher.py:54: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch [1/200], Step [100/3115], loss_D_A: 0.2194, loss_D_B: 0.2231, loss_G: 0.8609
Epoch [1/200], Step [200/3115], loss_D_A: 0.1753, loss_D_B: 0.2431, loss_G: 0.6738
Epoch [1/200], Step [300/3115], loss_D_A: 0.1441, loss_D_B: 0.1153, loss_G: 0.9124
Epoch [1/200], Step [400/3115], loss_D_A: 0.3136, loss_D_B: 0.2282, loss_G: 1.6230
Epoch [1/200], Step [500/3115], loss_D_A: 0.2813, loss_D_B: 0.2316, loss_G: 0.6413
Epoch [1/200], Step [600/3115], loss_D_A: 0.2713, loss_D_B: 0.1299, loss_G: 0.8426
Epoch [1/200], Step [700/3115], loss_D_A: 0.1320, loss_D_B: 0.4108, loss_G: 0.8317
Epoch [1/200], Step [800/3115], loss_D_A: 0.0586, loss_D_B: 0.2890, loss_G: 0.9109
Epoch [1/200], Step [900/3115], loss_D_A: 0.1838, loss_D_B: 0.1723, loss_G: 0.9038
Epoch [1/200], Step [1000/3115], loss_D_A: 0.1014, loss_D_B: 0.1265, loss_G: 0.8824
Epoch [1/200], Step [1100/3115], loss_D_A: 0.1553, loss_D_B: 0.4096, loss_G: 1.0517
Epoch [1/200], Step [1200/3115], loss_D_A: 0.0934, loss_D_B: 0.1643, loss_G: 0.8259
E

Epoch [4/200], Step [600/3115], loss_D_A: 0.0644, loss_D_B: 0.1350, loss_G: 1.3594
Epoch [4/200], Step [700/3115], loss_D_A: 0.1108, loss_D_B: 0.1281, loss_G: 0.8723
Epoch [4/200], Step [800/3115], loss_D_A: 0.1323, loss_D_B: 0.0653, loss_G: 1.0888
Epoch [4/200], Step [900/3115], loss_D_A: 0.0815, loss_D_B: 0.0162, loss_G: 1.7899
Epoch [4/200], Step [1000/3115], loss_D_A: 0.0715, loss_D_B: 0.2148, loss_G: 0.5540
Epoch [4/200], Step [1100/3115], loss_D_A: 0.0450, loss_D_B: 0.2143, loss_G: 0.7879
Epoch [4/200], Step [1200/3115], loss_D_A: 0.1164, loss_D_B: 0.2782, loss_G: 0.5366
Epoch [4/200], Step [1300/3115], loss_D_A: 0.1317, loss_D_B: 0.2533, loss_G: 0.6078
Epoch [4/200], Step [1400/3115], loss_D_A: 0.1840, loss_D_B: 0.1944, loss_G: 0.5942
Epoch [4/200], Step [1500/3115], loss_D_A: 0.2530, loss_D_B: 0.1653, loss_G: 0.6284
Epoch [4/200], Step [1600/3115], loss_D_A: 0.1574, loss_D_B: 0.1879, loss_G: 0.5319
Epoch [4/200], Step [1700/3115], loss_D_A: 0.0472, loss_D_B: 0.2794, loss_G: 0.5

Epoch [7/200], Step [1100/3115], loss_D_A: 0.0495, loss_D_B: 0.1428, loss_G: 0.9207
Epoch [7/200], Step [1200/3115], loss_D_A: 0.0147, loss_D_B: 0.2526, loss_G: 0.8407
Epoch [7/200], Step [1300/3115], loss_D_A: 0.1741, loss_D_B: 0.1896, loss_G: 0.7715
Epoch [7/200], Step [1400/3115], loss_D_A: 0.0678, loss_D_B: 0.2073, loss_G: 0.5023
Epoch [7/200], Step [1500/3115], loss_D_A: 0.1501, loss_D_B: 0.1766, loss_G: 1.2723
Epoch [7/200], Step [1600/3115], loss_D_A: 0.1132, loss_D_B: 0.2398, loss_G: 0.4684
Epoch [7/200], Step [1700/3115], loss_D_A: 0.0207, loss_D_B: 0.2804, loss_G: 0.7039
Epoch [7/200], Step [1800/3115], loss_D_A: 0.0221, loss_D_B: 0.1755, loss_G: 0.8753
Epoch [7/200], Step [1900/3115], loss_D_A: 0.0399, loss_D_B: 0.2507, loss_G: 1.0560
Epoch [7/200], Step [2000/3115], loss_D_A: 0.0402, loss_D_B: 0.1616, loss_G: 0.8031
Epoch [7/200], Step [2100/3115], loss_D_A: 0.0174, loss_D_B: 0.0729, loss_G: 1.1735
Epoch [7/200], Step [2200/3115], loss_D_A: 0.0793, loss_D_B: 0.1590, loss_G:

Epoch [10/200], Step [1600/3115], loss_D_A: 0.0136, loss_D_B: 0.1551, loss_G: 0.7772
Epoch [10/200], Step [1700/3115], loss_D_A: 0.0296, loss_D_B: 0.3722, loss_G: 0.6075
Epoch [10/200], Step [1800/3115], loss_D_A: 0.0039, loss_D_B: 0.2162, loss_G: 0.9790
Epoch [10/200], Step [1900/3115], loss_D_A: 0.0481, loss_D_B: 0.2519, loss_G: 1.0963
Epoch [10/200], Step [2000/3115], loss_D_A: 0.2141, loss_D_B: 0.1801, loss_G: 1.0685
Epoch [10/200], Step [2100/3115], loss_D_A: 0.0166, loss_D_B: 0.1922, loss_G: 0.9166
Epoch [10/200], Step [2200/3115], loss_D_A: 0.0102, loss_D_B: 0.1457, loss_G: 1.0506
Epoch [10/200], Step [2300/3115], loss_D_A: 0.0060, loss_D_B: 0.1124, loss_G: 1.0807
Epoch [10/200], Step [2400/3115], loss_D_A: 0.0052, loss_D_B: 0.1560, loss_G: 0.9422
Epoch [10/200], Step [2500/3115], loss_D_A: 0.0027, loss_D_B: 0.1110, loss_G: 1.1351
Epoch [10/200], Step [2600/3115], loss_D_A: 0.0053, loss_D_B: 0.2252, loss_G: 0.9458
Epoch [10/200], Step [2700/3115], loss_D_A: 0.0401, loss_D_B: 0.1

...

Epoch [192/200], Step [1600/3115], loss_D_A: 0.0000, loss_D_B: 0.0342, loss_G: 1.1766
Epoch [192/200], Step [1700/3115], loss_D_A: 0.0000, loss_D_B: 0.0133, loss_G: 1.0890
Epoch [192/200], Step [1800/3115], loss_D_A: 0.0000, loss_D_B: 0.0318, loss_G: 1.1216
Epoch [192/200], Step [1900/3115], loss_D_A: 0.0000, loss_D_B: 0.0297, loss_G: 1.1008
Epoch [192/200], Step [2000/3115], loss_D_A: 0.0000, loss_D_B: 0.0262, loss_G: 1.0887
Epoch [192/200], Step [2100/3115], loss_D_A: 0.0000, loss_D_B: 0.0164, loss_G: 1.1051
Epoch [192/200], Step [2200/3115], loss_D_A: 0.0000, loss_D_B: 0.0302, loss_G: 1.0334
Epoch [192/200], Step [2300/3115], loss_D_A: 0.0000, loss_D_B: 0.0406, loss_G: 1.2456
Epoch [192/200], Step [2400/3115], loss_D_A: 0.0000, loss_D_B: 0.0284, loss_G: 1.1051
Epoch [192/200], Step [2500/3115], loss_D_A: 0.0000, loss_D_B: 0.0306, loss_G: 1.1096
Epoch [192/200], Step [2600/3115], loss_D_A: 0.0001, loss_D_B: 0.0311, loss_G: 1.0637
Epoch [192/200], Step [2700/3115], loss_D_A: 0.0009, l

Epoch [195/200], Step [1900/3115], loss_D_A: 0.0000, loss_D_B: 0.0454, loss_G: 1.1280
Epoch [195/200], Step [2000/3115], loss_D_A: 0.0000, loss_D_B: 0.0190, loss_G: 1.0974
Epoch [195/200], Step [2100/3115], loss_D_A: 0.0001, loss_D_B: 0.0277, loss_G: 1.0909
Epoch [195/200], Step [2200/3115], loss_D_A: 0.0000, loss_D_B: 0.0276, loss_G: 1.1411
Epoch [195/200], Step [2300/3115], loss_D_A: 0.0000, loss_D_B: 0.0131, loss_G: 1.1403
Epoch [195/200], Step [2400/3115], loss_D_A: 0.0000, loss_D_B: 0.0320, loss_G: 1.1076
Epoch [195/200], Step [2500/3115], loss_D_A: 0.0000, loss_D_B: 0.0172, loss_G: 1.1763
Epoch [195/200], Step [2600/3115], loss_D_A: 0.0000, loss_D_B: 0.0320, loss_G: 0.9946
Epoch [195/200], Step [2700/3115], loss_D_A: 0.0000, loss_D_B: 0.0180, loss_G: 1.1646
Epoch [195/200], Step [2800/3115], loss_D_A: 0.0000, loss_D_B: 0.0115, loss_G: 1.1093
Epoch [195/200], Step [2900/3115], loss_D_A: 0.0000, loss_D_B: 0.0427, loss_G: 1.0584
Epoch [195/200], Step [3000/3115], loss_D_A: 0.0000, l

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7ff3dd76f940>>
Traceback (most recent call last):
  File "/home/ncp/anaconda3/envs/pytorch0_4/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/home/ncp/anaconda3/envs/pytorch0_4/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/ncp/anaconda3/envs/pytorch0_4/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/ncp/anaconda3/envs/pytorch0_4/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/home/ncp/anaconda3/envs/pytorch0_4/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/home/ncp/anaconda3

Epoch [196/200], Step [100/3115], loss_D_A: 0.0000, loss_D_B: 0.0188, loss_G: 1.1122
Epoch [196/200], Step [200/3115], loss_D_A: 0.0002, loss_D_B: 0.0505, loss_G: 1.1628
Epoch [196/200], Step [300/3115], loss_D_A: 0.0000, loss_D_B: 0.0290, loss_G: 1.0637
Epoch [196/200], Step [400/3115], loss_D_A: 0.0005, loss_D_B: 0.0160, loss_G: 1.2607
Epoch [196/200], Step [500/3115], loss_D_A: 0.0001, loss_D_B: 0.0321, loss_G: 1.0914
Epoch [196/200], Step [600/3115], loss_D_A: 0.0000, loss_D_B: 0.0208, loss_G: 1.1145
Epoch [196/200], Step [700/3115], loss_D_A: 0.0001, loss_D_B: 0.0375, loss_G: 1.2058
Epoch [196/200], Step [800/3115], loss_D_A: 0.0004, loss_D_B: 0.0308, loss_G: 1.0706
Epoch [196/200], Step [900/3115], loss_D_A: 0.0000, loss_D_B: 0.0195, loss_G: 1.1079
Epoch [196/200], Step [1000/3115], loss_D_A: 0.0000, loss_D_B: 0.0594, loss_G: 1.0534
Epoch [196/200], Step [1100/3115], loss_D_A: 0.0002, loss_D_B: 0.0185, loss_G: 1.2431
Epoch [196/200], Step [1200/3115], loss_D_A: 0.0000, loss_D_B: 

Epoch [199/200], Step [400/3115], loss_D_A: 0.0000, loss_D_B: 0.0318, loss_G: 0.9990
Epoch [199/200], Step [500/3115], loss_D_A: 0.0000, loss_D_B: 0.0111, loss_G: 1.1706
Epoch [199/200], Step [600/3115], loss_D_A: 0.0000, loss_D_B: 0.0108, loss_G: 1.1865
Epoch [199/200], Step [700/3115], loss_D_A: 0.0000, loss_D_B: 0.0423, loss_G: 1.0454
Epoch [199/200], Step [800/3115], loss_D_A: 0.0000, loss_D_B: 0.0255, loss_G: 1.1887
Epoch [199/200], Step [900/3115], loss_D_A: 0.0000, loss_D_B: 0.0196, loss_G: 1.0732
Epoch [199/200], Step [1000/3115], loss_D_A: 0.0000, loss_D_B: 0.0523, loss_G: 0.9303
Epoch [199/200], Step [1100/3115], loss_D_A: 0.0000, loss_D_B: 0.0239, loss_G: 1.2058
Epoch [199/200], Step [1200/3115], loss_D_A: 0.0000, loss_D_B: 0.0178, loss_G: 1.0785
Epoch [199/200], Step [1300/3115], loss_D_A: 0.0000, loss_D_B: 0.0265, loss_G: 1.1318
Epoch [199/200], Step [1400/3115], loss_D_A: 0.0000, loss_D_B: 0.0334, loss_G: 1.0977
Epoch [199/200], Step [1500/3115], loss_D_A: 0.0000, loss_D_